# 국민청원 데이터 가져오기


- 참고:
  - [국민청원 데이터셋 CSV](https://newhiwoong.github.io/%EA%B5%AD%EB%AF%BC%EC%B2%AD%EC%9B%90/%EA%B5%AD%EB%AF%BC%EC%B2%AD%EC%9B%90-%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B)
  - [청와대 국민청원 https://www1.president.go.kr/petitions](https://www1.president.go.kr/petitions)
  - [Using Selenium with Google Colaboratory https://darektidwell.com/using-selenium-with-google-colaboratory/](https://darektidwell.com/using-selenium-with-google-colaboratory/)
  - [https://stackoverflow.com/questions/56829470/selenium-google-colab-error-chromedriver-executable-needs-to-be-in-path](https://stackoverflow.com/questions/56829470/selenium-google-colab-error-chromedriver-executable-needs-to-be-in-path)
 
- 참고링크 중 "국민청원 데이터셋 CSV"의 내용을 토대로 수정하여 작성
- 1건당 1-2초(google colab에서 더 느림, 실 사용은 로컬을 권장함)

## 확인된 에러의 경우 
1. 없는 페이지
2. 관리자 차단 페이지

In [3]:
import sys

# install chromium, its driver, and selenium
if 'google.colab' in sys.modules:
    !apt-get update
    !apt install chromium-chromedriver
    !cp /usr/lib/chromium-browser/chromedriver /usr/bin
    !pip install selenium

# set options to be headless, ..
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import time
import re

if 'ipykernel' in sys.modules:
    from tqdm import tqdm_notebook as tqdm
else:
    from tqdm import tqdm

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [4]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [9]:
def get_petition(wd, number):
    base_url='https://www1.president.go.kr/petitions/'
    try:
        wd.get(base_url+str(number))
        element = WebDriverWait(wd, 2).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'petitionsView_title')))
        #bs = BeautifulSoup(wd.page_source, 'html.parser')
        bs = BeautifulSoup(wd.page_source, 'lxml')
        progress = bs.find('div',{'class':'petitionsView_progress'}).get_text().strip()
        title = bs.find('h3',{'class':'petitionsView_title'}).get_text().strip()
        like = bs.find('h2',{'class':'petitionsView_count'}).get_text().strip()
        like = int(''.join(re.findall('\d+', str(like))))

        if progress == '답변완료':
            contlist = bs.find_all('div',{'class':'View_write'})
            len_contlist = len(bs.find_all('div',{'class':'View_write'}))
            contents = bs.find_all('div',{'class':'View_write'})[len_contlist-1].get_text().strip()
            answer = bs.find_all('div',{'class':'pr_tk25'})[len_contlist-1].get_text().strip()[4:].strip()
        else:
            contents = bs.find('div',{'class':'View_write'}).get_text().strip()
            answer = ''

        info = bs.find('ul',{'class':'petitionsView_info_list'}).get_text().split('\n')
        category = info[1][4:]
        sday = info[2][4:]
        eday = info[3][4:]
        proponent = info[4][3:].split()[0]
        return {'number':number, 'progress':progress, 'title':title,  'like':like,
                'contents':contents, 'answer':answer, 'category':category,
                'sday':sday, 'eday':eday, 'proponent':proponent}
    except Exception as e:
        print(number, e)
        return {'number':number, 'progress':'error', 'title':'',  'like':0,
                'contents':str(e), 'answer':'', 'category':'',
                'sday':'', 'eday':'', 'proponent':''}

In [7]:
if __name__ == '__main__' or 'google.colab' in sys.modules:
    wd = webdriver.Chrome('chromedriver',options=options)
    print(get_petition(wd, 598683))
    print(get_petition(wd, 582349))
    wd.quit()

point 1
한강 실종 대학생 *** 학생의 억울한 죽음에 대한 진상 규명 부탁드립니다.

한강 실종 대학생 ***군의 사망사건 모르는사람이 없을 듯 합니다. 너무 너무 안타까운 사건인데요.

이 학생의 죽음은 사고가 아닌 사건인듯 합니다.
누가들어도 이상한 ***군과 같이있던 친구의 진술, 그리고 경찰측에서는 제일 사건사고에 연관성이 있어보이는듯한 친구는 조사를 하지않고, 목격자만 찾고있다고하네요.

현재 그 친구와 부모님은..
핸드폰 제출도 거부하며, 장례식장에도 나타나지 않았으며, 아파트 씨씨티비공개거부, 차량블랙박스도 공개 거부하는 상태라고합니다.
그 날 신고있던 운동화도 버렸다고하는데요.


누가봐도 이상한 이 사건, 왜 경찰은 그 시간대까지 같이 술먹고 함께있던 ***군의 친구는 조사를 하지않고, 목격자만 찾고있는 확실한 진실 규명이 필요합니다. 고 ***군과 남아있는 ***군의 부모님의 억울함을 풀어주세요.

 [본 게시물의 일부 내용이 국민 청원 요건에 위배되어 관리자에 의해 수정되었습니다]
point 3
point 4
{'number': 598683, 'progress': '답변완료', 'title': '한강 실종 대학생 고 ***군의 억울함을 풀어주세요.', 'like': 521867, 'contents': '한강 실종 대학생 *** 학생의 억울한 죽음에 대한 진상 규명 부탁드립니다.\n\n한강 실종 대학생 ***군의 사망사건 모르는사람이 없을 듯 합니다. 너무 너무 안타까운 사건인데요.\n\n이 학생의 죽음은 사고가 아닌 사건인듯 합니다.\n누가들어도 이상한 ***군과 같이있던 친구의 진술, 그리고 경찰측에서는 제일 사건사고에 연관성이 있어보이는듯한 친구는 조사를 하지않고, 목격자만 찾고있다고하네요.\n\n현재 그 친구와 부모님은..\n핸드폰 제출도 거부하며, 장례식장에도 나타나지 않았으며, 아파트 씨씨티비공개거부, 차량블랙박스도 공개 거부하는 상태라고합니다.\n그 날 신고있던 운동화도 버렸다고하는데요.\n\n\n누가봐도 이상한 이 사건,

## 사용예

In [11]:
import pandas as pd

nums_list = [598683, 598794, 598497, 598424, 598340, 598291, 598239, 598232, 597991, 597851, 598240, 598123, 597775, 597854, 597687, 597490, 597240, 597744, 597214, 597157, 595180, 597220, 595551, 595630, 595595, 595454, 595214, 594987, 594995, 594944, 594707, 594474, 594746, 594410, 594259, 594227, 595305, 595321, 595304, 595310, 594102, 594108, 594063, 593979, 593728, 593519, 594131, 593228, 593230, 592978, 592804, 593227, 592340, 592609, 592173, 592153, 591978, 591956, 592276, 592084, 591700, 591675, 591808, 590970, 590966, 590879, 591074, 590746, 590550, 590504, 590416, 590341, 589941, 589634, 590258, 588752, 588455, 588340, 587827, 587624, 587604, 587334, 587169, 586926, 587323, 587335, 587130, 586941, 586860, 585007, 586878, 587352, 586650, 586609, 585926, 585665, 586110, 585083, 584936, 585683, 585682, 585795, 585290, 585610, 586819, 586880, 586885, 586879, 586522, 584593, 584755, 584335, 584284, 584204, 583841, 583605, 583274, 583677, 583122, 583346, 583543, 583272, 583277, 583190, 583076, 583046, 582785, 582620, 582349, 582351, 582301, 581667, 582070, 582019, 582190, 581808, 581713, 581380, 581384, 581300, 581465, 580884, 580402, 580452, 579523, 580325, 580707, 580639, 579840, 579612, 579708, 579682, 579779, 579927, 579401, 579585, 577697, 570236, 561699, 579353, 579426, 541615, 534182, 578761, 410524, 559071, 553263, 511075, 510422, 527553, 531087, 530372, 512112, 489302, 522031, 472309, 408154, 462340, 454045, 449519, 462618, 462623, 437737, 442093, 446686, 426349, 436901, 418376, 412722, 411485, 410447, 445550, 414891, 426834, 408609, 417147, 398030, 384676, 397558, 395955, 369690, 322420, 314584, 294226, 282903, 281200, 272632, 284845, 273112, 269548, 242950, 246489, 244045, 270921, 236397, 231701, 199622, 219348, 199778, 201953, 187633, 178781, 235102, 173355, 230552, 174292, 164255, 203181, 165796, 148613, 157046, 151430, 141333, 134590, 148806, 113699, 95138, 101349, 96427, 105105, 142600, 79162, 81026, 127921, 76020, 57132, 49083, 45581, 10949, 26024, 18278, 1785]

def get_p(num = 583432, cnt = 10):
    rtn = []
    wd = webdriver.Chrome('chromedriver',options=options)
    # for idx, i in enumerate(tqdm(range(num-cnt, num))):
    for idx, i in enumerate(tqdm(nums_list)):
        tmp = get_petition(wd, i)
        if idx == 0:
            rtn_c = list(tmp.keys())
        rtn.append(list(tmp.values()))
        # 오래 유지될경우 느려지는 경향이 있어 중간에 초기화
        if idx % 200 == 0:
            wd.quit()
            wd = webdriver.Chrome('chromedriver',options=options)
    # for idx, i in enumerate(nums_list):
    #     print(i)
    #     tmp = get_petition(wd, i)
    #     if idx == 0:
    #         rtn_c = list(tmp.keys())
    #     rtn.append(list(tmp.values()))
    #     # 오래 유지될경우 느려지는 경향이 있어 중간에 초기화
    #     if idx % 200 == 0:
    #         wd.quit()
    #         wd = webdriver.Chrome('chromedriver',options=options)

    return (rtn_c, rtn)


c, rtn = get_p()
%time wd = webdriver.Chrome('chromedriver',options=options)
# %time rtn.append(list(get_petition(wd, 582349).values()))
# %time rtn.append(list(get_petition(wd, 10).values()))
wd.quit()
result = pd.DataFrame(rtn, columns=c)
result.to_csv("petitions.csv", encoding='utf-8-sig')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


436901 list index out of range
418376 list index out of range
412722 list index out of range

CPU times: user 12.5 ms, sys: 6.1 ms, total: 18.6 ms
Wall time: 1.27 s


[https://www1.president.go.kr/petitions/10](https://www1.president.go.kr/petitions/10)
- 관리자에 의해 비공개된 청원입니다